# 🤖 Blockchain Fraud Detection - Machine Learning Experiments

This notebook experiments with machine learning models for detecting fraudulent wallet behavior on the Ethereum blockchain.

## Objectives
1. **Feature Engineering**: Compute wallet-level features from transaction data
2. **Model Training**: Train anomaly detection models (Isolation Forest, LOF, DBSCAN)
3. **Evaluation**: Analyze model performance and interpret results
4. **Save Results**: Store features and scores to BigQuery for production use

## Models Used
- **Isolation Forest**: Tree-based anomaly detection
- **Local Outlier Factor (LOF)**: Density-based outlier detection
- **DBSCAN**: Clustering-based anomaly detection
- **Ensemble**: Weighted combination of all models


## 1. Setup and Configuration


In [ ]:
# Install required packages (run once if needed)
# Use %pip for better Jupyter compatibility, especially on Windows
# Uncomment the line below to install packages
# %pip install google-cloud-bigquery pandas numpy scikit-learn matplotlib seaborn db-dtypes pyarrow python-dotenv


IndentationError: unexpected indent (4232343709.py, line 4)

In [ ]:
# Import libraries
import os
import sys
import warnings
from datetime import datetime
from typing import Tuple, Dict, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score

from google.cloud import bigquery
from dotenv import load_dotenv

# Suppress warnings
warnings.filterwarnings('ignore')

# Configure display
pd.set_option('display.max_columns', 50)

# Matplotlib style - use fallback for compatibility
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except OSError:
    try:
        plt.style.use('seaborn-darkgrid')
    except OSError:
        plt.style.use('ggplot')  # Fallback to ggplot
        
plt.rcParams['figure.figsize'] = (12, 6)

print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")


In [ ]:
# Load environment and configure BigQuery
load_dotenv()

PROJECT_ID = os.getenv('GOOGLE_CLOUD_PROJECT', 'blockchain-481614')
RAW_DATASET = 'blockchain_raw'
ML_DATASET = 'blockchain_ml'

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

def run_query(query: str) -> pd.DataFrame:
    """Execute BigQuery query and return DataFrame."""
    return client.query(query).to_dataframe()

print(f"✅ Project: {PROJECT_ID}")
print(f"✅ Raw Dataset: {RAW_DATASET}")
print(f"✅ ML Dataset: {ML_DATASET}")


## 2. Feature Engineering

Compute wallet-level features from raw transaction data. These features capture various aspects of wallet behavior that may indicate fraudulent activity.


In [ ]:
# Feature engineering SQL query
# This computes wallet-level features from transaction data

features_query = f"""
WITH wallet_transactions AS (
    -- Get all transactions with wallet as sender
    SELECT
        from_address AS wallet_address,
        CAST(value_eth AS FLOAT64) as value_eth,
        CAST(gas_price AS FLOAT64) as gas_price,
        CAST(gas_used AS INT64) as gas_used,
        to_address AS counterparty,
        transaction_timestamp,
        'out' AS direction
    FROM `{PROJECT_ID}.{RAW_DATASET}.raw_transactions`
    WHERE from_address IS NOT NULL
    
    UNION ALL
    
    -- Get all transactions with wallet as receiver
    SELECT
        to_address AS wallet_address,
        CAST(value_eth AS FLOAT64) as value_eth,
        CAST(gas_price AS FLOAT64) as gas_price,
        CAST(gas_used AS INT64) as gas_used,
        from_address AS counterparty,
        transaction_timestamp,
        'in' AS direction
    FROM `{PROJECT_ID}.{RAW_DATASET}.raw_transactions`
    WHERE to_address IS NOT NULL
),

basic_features AS (
    SELECT
        wallet_address,
        
        -- Transaction counts
        COUNT(*) AS tx_count,
        COUNTIF(direction = 'in') AS tx_count_in,
        COUNTIF(direction = 'out') AS tx_count_out,
        
        -- Value statistics
        SUM(value_eth) AS total_value,
        SUM(CASE WHEN direction = 'in' THEN value_eth ELSE 0 END) AS total_value_in,
        SUM(CASE WHEN direction = 'out' THEN value_eth ELSE 0 END) AS total_value_out,
        AVG(value_eth) AS avg_value,
        STDDEV(value_eth) AS std_value,
        MIN(value_eth) AS min_value,
        MAX(value_eth) AS max_value,
        
        -- Counterparty analysis
        COUNT(DISTINCT counterparty) AS unique_counterparties,
        
        -- Gas statistics
        AVG(gas_used) AS avg_gas_used,
        AVG(gas_price / 1e9) AS avg_gas_price_gwei,
        
        -- Temporal
        MIN(transaction_timestamp) AS first_tx_time,
        MAX(transaction_timestamp) AS last_tx_time,
        TIMESTAMP_DIFF(MAX(transaction_timestamp), MIN(transaction_timestamp), DAY) AS activity_span_days,
        COUNT(DISTINCT DATE(transaction_timestamp)) AS active_days
        
    FROM wallet_transactions
    GROUP BY wallet_address
    HAVING COUNT(*) >= 2  -- Minimum 2 transactions
)

SELECT
    wallet_address,
    tx_count,
    tx_count_in,
    tx_count_out,
    total_value,
    total_value_in,
    total_value_out,
    avg_value,
    COALESCE(std_value, 0) as std_value,
    min_value,
    max_value,
    unique_counterparties,
    avg_gas_used,
    avg_gas_price_gwei,
    activity_span_days,
    active_days,
    
    -- Derived features
    SAFE_DIVIDE(tx_count_in, tx_count_out) AS in_out_ratio,
    total_value_in - total_value_out AS net_flow,
    SAFE_DIVIDE(tx_count, GREATEST(active_days, 1)) AS tx_per_active_day,
    SAFE_DIVIDE(total_value, tx_count) AS value_per_tx
    
FROM basic_features
WHERE wallet_address IS NOT NULL
"""

print("Running feature engineering query...")
features_df = run_query(features_query)
print(f"✅ Computed features for {len(features_df):,} wallets")
print(f"✅ Total features: {len(features_df.columns) - 1}")  # -1 for wallet_address


In [ ]:
# Explore the features
print("Feature Columns:")
print(list(features_df.columns))
print("\nSample Data:")
display(features_df.head())


In [ ]:
# Feature statistics
print("📊 Feature Statistics:")
display(features_df.describe())


## 3. Data Preparation

Prepare the data for machine learning by handling missing values and scaling features.


In [ ]:
# Prepare features for ML
# Get numeric columns only (exclude wallet_address)
feature_cols = [col for col in features_df.columns if col != 'wallet_address']
X = features_df[feature_cols].copy()

# Handle missing and infinite values
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0)

print(f"Feature matrix shape: {X.shape}")
print(f"Missing values: {X.isna().sum().sum()}")

# Store wallet addresses for later
wallet_addresses = features_df['wallet_address'].values


In [ ]:
# Scale features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Scaled feature matrix shape: {X_scaled.shape}")
print(f"Feature means (should be ~0): {X_scaled.mean(axis=0)[:5].round(4)}")
print(f"Feature stds (should be ~1): {X_scaled.std(axis=0)[:5].round(4)}")


## 4. Model Training

Train multiple anomaly detection models to identify potentially fraudulent wallets.

### 4.1 Isolation Forest
Isolation Forest isolates anomalies by randomly selecting features and split values. Anomalies are easier to isolate and thus have shorter path lengths.


In [ ]:
# Train Isolation Forest
print("Training Isolation Forest...")

iso_forest = IsolationForest(
    n_estimators=100,
    contamination=0.1,  # Expected 10% anomalies
    max_samples='auto',
    random_state=42,
    n_jobs=-1
)

# Fit and predict
iso_labels = iso_forest.fit_predict(X_scaled)
iso_scores = iso_forest.decision_function(X_scaled)

# Convert labels: -1 (anomaly) -> 1, 1 (normal) -> 0
iso_anomaly = (iso_labels == -1).astype(int)

print(f"✅ Isolation Forest trained")
print(f"   Anomalies detected: {iso_anomaly.sum():,} ({iso_anomaly.mean()*100:.1f}%)")


### 4.2 Local Outlier Factor (LOF)
LOF measures the local density deviation of a data point with respect to its neighbors. Points with substantially lower density than their neighbors are considered outliers.


In [ ]:
# Train Local Outlier Factor
print("Training Local Outlier Factor...")

lof = LocalOutlierFactor(
    n_neighbors=20,
    contamination=0.1,
    n_jobs=-1
)

# Fit and predict
lof_labels = lof.fit_predict(X_scaled)
lof_scores = -lof.negative_outlier_factor_  # Negate to make higher = more anomalous

# Convert labels
lof_anomaly = (lof_labels == -1).astype(int)

print(f"✅ LOF trained")
print(f"   Anomalies detected: {lof_anomaly.sum():,} ({lof_anomaly.mean()*100:.1f}%)")


### 4.3 DBSCAN
DBSCAN clusters data points based on density. Points that don't belong to any cluster (noise points) are considered anomalies.


In [ ]:
# Train DBSCAN
print("Training DBSCAN...")

dbscan = DBSCAN(
    eps=0.5,
    min_samples=5,
    n_jobs=-1
)

# Fit and predict
dbscan_labels = dbscan.fit_predict(X_scaled)

# Noise points (label -1) are anomalies
dbscan_anomaly = (dbscan_labels == -1).astype(int)

# Calculate silhouette score (excluding noise)
non_noise_mask = dbscan_labels != -1
if len(np.unique(dbscan_labels[non_noise_mask])) > 1:
    sil_score = silhouette_score(X_scaled[non_noise_mask], dbscan_labels[non_noise_mask])
else:
    sil_score = 0

print(f"✅ DBSCAN trained")
print(f"   Clusters found: {len(np.unique(dbscan_labels)) - 1}")  # -1 for noise
print(f"   Noise points (anomalies): {dbscan_anomaly.sum():,} ({dbscan_anomaly.mean()*100:.1f}%)")
print(f"   Silhouette Score: {sil_score:.3f}")


### 4.4 Ensemble Model
Combine all models into an ensemble score for more robust fraud detection.


In [ ]:
# Create ensemble fraud score
def normalize_scores(scores):
    """Normalize scores to 0-1 range."""
    min_s = scores.min()
    max_s = scores.max()
    if max_s == min_s:
        return np.zeros_like(scores)
    return (scores - min_s) / (max_s - min_s)

# Normalize individual scores
iso_norm = normalize_scores(-iso_scores)  # Negate because lower = more anomalous
lof_norm = normalize_scores(lof_scores)
dbscan_norm = dbscan_anomaly.astype(float)

# Weighted ensemble (higher weight for Isolation Forest)
weights = {'isolation_forest': 0.4, 'lof': 0.35, 'dbscan': 0.25}
ensemble_score = (
    weights['isolation_forest'] * iso_norm +
    weights['lof'] * lof_norm +
    weights['dbscan'] * dbscan_norm
)

print("✅ Ensemble scores computed")
print(f"   Score range: {ensemble_score.min():.3f} - {ensemble_score.max():.3f}")
print(f"   Mean score: {ensemble_score.mean():.3f}")


## 5. Model Evaluation and Visualization

Analyze the model results and visualize the fraud score distribution.


In [ ]:
# Assign risk categories
def get_risk_category(score):
    """Assign risk category based on fraud score."""
    if score >= 0.9:
        return 'critical'
    elif score >= 0.7:
        return 'high'
    elif score >= 0.4:
        return 'medium'
    else:
        return 'low'

risk_categories = np.array([get_risk_category(s) for s in ensemble_score])

# Count by category
from collections import Counter
risk_counts = Counter(risk_categories)
print("📊 Risk Category Distribution:")
for cat in ['low', 'medium', 'high', 'critical']:
    count = risk_counts.get(cat, 0)
    pct = count / len(risk_categories) * 100
    print(f"   {cat.capitalize()}: {count:,} ({pct:.1f}%)")


In [ ]:
# Visualize fraud score distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Ensemble score histogram
ax1 = axes[0, 0]
ax1.hist(ensemble_score, bins=50, edgecolor='black', alpha=0.7, color='#e74c3c')
ax1.axvline(x=0.4, color='orange', linestyle='--', linewidth=2, label='Medium Risk (0.4)')
ax1.axvline(x=0.7, color='red', linestyle='--', linewidth=2, label='High Risk (0.7)')
ax1.axvline(x=0.9, color='purple', linestyle='--', linewidth=2, label='Critical (0.9)')
ax1.set_xlabel('Ensemble Fraud Score')
ax1.set_ylabel('Frequency')
ax1.set_title('Ensemble Fraud Score Distribution')
ax1.legend()

# 2. Individual model comparison
ax2 = axes[0, 1]
ax2.boxplot([iso_norm, lof_norm, dbscan_norm], labels=['Isolation Forest', 'LOF', 'DBSCAN'])
ax2.set_ylabel('Normalized Score')
ax2.set_title('Model Score Comparison')

# 3. Risk category pie chart
ax3 = axes[1, 0]
colors_map = {'low': '#2ecc71', 'medium': '#f39c12', 'high': '#e74c3c', 'critical': '#8e44ad'}
categories = ['low', 'medium', 'high', 'critical']
counts = [risk_counts.get(c, 0) for c in categories]
colors = [colors_map[c] for c in categories]
ax3.pie(counts, labels=categories, autopct='%1.1f%%', colors=colors)
ax3.set_title('Risk Category Distribution')

# 4. Score percentiles
ax4 = axes[1, 1]
percentiles = [10, 25, 50, 75, 90, 95, 99]
pct_values = [np.percentile(ensemble_score, p) for p in percentiles]
ax4.bar([str(p) + '%' for p in percentiles], pct_values, color='#3498db', alpha=0.8)
ax4.set_xlabel('Percentile')
ax4.set_ylabel('Fraud Score')
ax4.set_title('Fraud Score Percentiles')
ax4.axhline(y=0.7, color='red', linestyle='--', label='High Risk Threshold')
ax4.legend()

plt.tight_layout()
plt.savefig('ml_fraud_detection_results.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Show top suspicious wallets
results_df = pd.DataFrame({
    'wallet_address': wallet_addresses,
    'fraud_score': ensemble_score,
    'risk_category': risk_categories,
    'isolation_forest_score': iso_norm,
    'lof_score': lof_norm,
    'dbscan_is_noise': dbscan_anomaly
})

# Merge with original features
results_df = results_df.merge(features_df, on='wallet_address', how='left')

# Top high-risk wallets
print("🚨 Top 10 High-Risk Wallets:")
high_risk_df = results_df[results_df['risk_category'].isin(['high', 'critical'])].sort_values('fraud_score', ascending=False)
display(high_risk_df[['wallet_address', 'fraud_score', 'risk_category', 'tx_count', 'total_value']].head(10))


## 6. Feature Importance Analysis

Analyze which features contribute most to anomaly detection.


In [ ]:
# Compare high-risk vs normal wallet features
high_risk_wallets = results_df[results_df['risk_category'].isin(['high', 'critical'])]
normal_wallets = results_df[results_df['risk_category'] == 'low']

print("📊 Feature Comparison: High-Risk vs Normal Wallets\n")
comparison_features = ['tx_count', 'total_value', 'avg_value', 'unique_counterparties', 
                       'activity_span_days', 'tx_per_active_day']

for feat in comparison_features:
    if feat in results_df.columns:
        hr_mean = high_risk_wallets[feat].mean()
        nr_mean = normal_wallets[feat].mean()
        ratio = hr_mean / nr_mean if nr_mean > 0 else float('inf')
        print(f"{feat}:")
        print(f"   High-Risk Mean: {hr_mean:,.2f}")
        print(f"   Normal Mean: {nr_mean:,.2f}")
        print(f"   Ratio: {ratio:.2f}x")
        print()


## 7. Save Results to BigQuery

Save the computed features and fraud scores to BigQuery for use by the production system.


In [ ]:
# Prepare data for BigQuery upload
# Features table
features_to_save = features_df.copy()
features_to_save['computed_at'] = datetime.utcnow()

print(f"Features to save: {len(features_to_save):,} rows")

# Fraud scores table
scores_to_save = pd.DataFrame({
    'wallet_address': wallet_addresses,
    'fraud_score': ensemble_score,
    'risk_category': risk_categories,
    'isolation_forest_score': iso_norm,
    'lof_score': lof_norm,
    'dbscan_is_noise': dbscan_anomaly.astype(bool),
    'scored_at': datetime.utcnow()
})

print(f"Scores to save: {len(scores_to_save):,} rows")


In [ ]:
# Function to save DataFrame to BigQuery
def save_to_bigquery(df: pd.DataFrame, dataset: str, table: str, write_mode: str = 'WRITE_TRUNCATE'):
    """Save DataFrame to BigQuery table."""
    table_id = f"{PROJECT_ID}.{dataset}.{table}"
    
    # Create dataset if it doesn't exist
    dataset_ref = f"{PROJECT_ID}.{dataset}"
    try:
        client.get_dataset(dataset_ref)
    except Exception:
        ds = bigquery.Dataset(dataset_ref)
        ds.location = "US"
        client.create_dataset(ds)
        print(f"Created dataset: {dataset_ref}")
    
    # Configure job
    job_config = bigquery.LoadJobConfig(
        write_disposition=write_mode
    )
    
    # Upload
    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
    job.result()  # Wait for completion
    
    print(f"✅ Saved {len(df):,} rows to {table_id}")
    return table_id


In [ ]:
# Save features and scores to BigQuery
# Uncomment the lines below to actually save to BigQuery

# Save wallet features
# save_to_bigquery(features_to_save, ML_DATASET, 'wallet_features')

# Save fraud scores
# save_to_bigquery(scores_to_save, ML_DATASET, 'wallet_fraud_scores')

print("💡 To save results to BigQuery, uncomment the save_to_bigquery() calls above.")


## 8. Summary

### Model Performance Summary


In [ ]:
# Final summary
print("="*60)
print("🤖 ML FRAUD DETECTION SUMMARY")
print("="*60)

print(f"\n📊 DATA:")
print(f"   Wallets analyzed: {len(wallet_addresses):,}")
print(f"   Features computed: {len(feature_cols)}")

print(f"\n🔍 MODELS:")
print(f"   Isolation Forest anomalies: {iso_anomaly.sum():,} ({iso_anomaly.mean()*100:.1f}%)")
print(f"   LOF anomalies: {lof_anomaly.sum():,} ({lof_anomaly.mean()*100:.1f}%)")
print(f"   DBSCAN noise points: {dbscan_anomaly.sum():,} ({dbscan_anomaly.mean()*100:.1f}%)")

print(f"\n🎯 ENSEMBLE RESULTS:")
for cat in ['low', 'medium', 'high', 'critical']:
    count = risk_counts.get(cat, 0)
    pct = count / len(risk_categories) * 100
    print(f"   {cat.capitalize()}: {count:,} ({pct:.1f}%)")

print(f"\n📁 OUTPUTS:")
print(f"   - ml_fraud_detection_results.png")
print(f"   - wallet_features table (ready for BigQuery)")
print(f"   - wallet_fraud_scores table (ready for BigQuery)")

print("\n" + "="*60)


## 📚 Next Steps

1. **Hyperparameter Tuning**: Experiment with different model parameters
2. **Feature Engineering**: Add more behavioral features (time-based, network-based)
3. **Model Comparison**: Try other algorithms (One-Class SVM, Autoencoders)
4. **Threshold Optimization**: Fine-tune risk category thresholds based on business requirements
5. **Production Deployment**: Schedule regular model retraining via the `data_science` pipeline

---

### Key Findings

- **Transaction Volume**: High-risk wallets typically have significantly higher transaction volumes
- **Value Patterns**: Anomalous wallets often show unusual value distributions (very high or clustered values)
- **Behavioral Indicators**: Concentrated counterparty interactions and unusual timing patterns are strong fraud indicators

### Model Considerations

- **Isolation Forest** works well for detecting isolated outliers
- **LOF** captures local density anomalies
- **DBSCAN** identifies wallets that don't fit any cluster pattern
- **Ensemble** provides more robust detection by combining multiple perspectives
